In [1]:
from dotenv import load_dotenv

load_dotenv(
    dotenv_path="/Users/tomwattley/App/racing-api-project/racing-api-project/libraries/api-helpers/src/api_helpers/.env"
)
from api_helpers.config import Config
import pandas as pd
import requests
import psycopg2
config = Config()

In [2]:
from api_helpers.clients import get_betfair_client, get_postgres_client

# bf_client = get_betfair_client()
pg_client = get_postgres_client()
import requests

2025-11-22T05:18:45Z | INFO - Logging configuration initialized with level: INFO


In [ ]:

API_TOKEN=''
BASE_URL='https://api.sportmonks.com/v3/football/'
DB_CONFIG = {
    "dbname": "football-api",
    "user": "postgres",
    "password": "postgres",
    "host": "localhost",
    "port": 5432
}

MY_LEAGUES = {
    'Premier League': 8,
    'Championship': 9,
    'League One': 12,
    'League Two': 14
}

def get_db():
    return psycopg2.connect(**DB_CONFIG)

In [4]:


# --- CONFIGURATION ---


# --- ⚡️ THE PAGINATION HELPER (GENERATOR) ---
def get_paginated(endpoint, params=None):
    """
    Yields items one by one from all pages.
    Handles the SportMonks next_page logic automatically.
    """
    if params is None:
        params = {}
    
    # Always attach the token
    params['api_token'] = API_TOKEN
    
    # Construct initial URL
    url = f"https://api.sportmonks.com/v3/football/{endpoint}"
    
    while url:
        print(f"      📡 Fetching page: {url} ...")
        try:
            # We pass params only if they aren't already in the URL (next_page has them)
            if 'api_token=' in url:
                resp = requests.get(url)
            else:
                resp = requests.get(url, params=params)

                
            resp.raise_for_status()
            data = resp.json()
            
            # 1. YIELD the data from this page
            # This passes items back to the main loop one by one
            for item in data.get('data', []):
                yield item
            
            # 2. CHECK for next page
            pagination = data.get('pagination', {})
            if pagination.get('has_more'):
                url = pagination['next_page']
                # Important: SportMonks next_page URL often needs the token re-appended
                if 'api_token' not in url:
                    url += f"&api_token={API_TOKEN}"
            else:
                url = None
                
        except Exception as e:
            print(f"      ❌ Error: {e}")
            break


In [8]:
# BASE_URL = "https://api.sportmonks.com/v3/football"

# # ⚠️ Make sure you have re-run the cell with your 'get_paginated' function first!

# def get_coaches():
#     conn = get_db()
#     cur = conn.cursor()

#     iterator = get_paginated(
#         "coaches", 
#         params={
#             'api_token':API_TOKEN,
#             "select": "id,player_id,country_id,nationality_id,name,display_name"
#         },
#     )

#     count = 0
#     for coach in iterator:
#         cur.execute(
#             """
#             INSERT INTO reference.coaches (
#                 id, 
#                 player_id, 
#                 country_id, 
#                 nationality_id,    -- Added this
#                 name, 
#                 display_name, 
#                 last_updated
#             )
#             VALUES (%s, %s, %s, %s, %s, %s, NOW()) -- 6 placeholders for the 6 fields below
#             ON CONFLICT (id) DO UPDATE SET
#                 player_id = EXCLUDED.player_id,
#                 country_id = EXCLUDED.country_id,
#                 nationality_id = EXCLUDED.nationality_id,
#                 name = EXCLUDED.name,
#                 display_name = EXCLUDED.display_name,
#                 last_updated = NOW();
#             """,
#             (
#                 coach.get("id"),
#                 coach.get("player_id"),
#                 coach.get("country_id"),
#                 coach.get("nationality_id"),
#                 coach.get("name"), 
#                 coach.get("display_name"),
#             ),
#         )
#         count += 1

#     print(f"      -> Saved {count} coaches")

#     conn.commit()
#     conn.close()
#     print("✅ Coaches Done.\n")

# get_coaches()

In [10]:
# def get_players():
#     conn = get_db()
#     cur = conn.cursor()

#     iterator = get_paginated(
#         "players", 
#         params={
#             'api_token':API_TOKEN,
#             "select": "id,country_id,nationality_id,city_id,position_id,detailed_position_id,type_id,name,display_name"
#         },
#     )

#     count = 0
#     for player in iterator:
#         cur.execute(
#             """
#             INSERT INTO reference.players (
#                 id, 
#                 country_id, 
#                 nationality_id,
#                 city_id,
#                 position_id,
#                 detailed_position_id,
#                 type_id,
#                 name, 
#                 display_name, 
#                 last_updated
#             )
#             VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, NOW())
#             ON CONFLICT (id) DO UPDATE SET
#                 country_id = EXCLUDED.country_id,
#                 nationality_id = EXCLUDED.nationality_id,
#                 city_id = EXCLUDED.city_id,
#                 position_id = EXCLUDED.position_id,
#                 detailed_position_id = EXCLUDED.detailed_position_id,
#                 type_id = EXCLUDED.type_id,
#                 name = EXCLUDED.name,
#                 display_name = EXCLUDED.display_name,
#                 last_updated = NOW();
#             """,
#             (
#                 player.get("id"),
#                 player.get("country_id"),
#                 player.get("nationality_id"),
#                 player.get("city_id"),
#                 player.get("position_id"),
#                 player.get("detailed_position_id"),
#                 player.get("type_id"),
#                 player.get("name"), 
#                 player.get("display_name"),
#             ),
#         )
#         count += 1

#     print(f"      -> Saved {count} players")

#     conn.commit()
#     conn.close()
#     print("✅ Players Done.\n")

# get_players()

In [14]:

# def sync_teams_in_season(season_id, league_id):
#     """
#     Fetches teams for a specific season and saves to reference.season_teams
#     """
#     conn = get_db()
#     cur = conn.cursor()

#     print(f"   ... Syncing Season {season_id} (League {league_id})")

#     # Call SportMonks: Get all teams for this season ID
#     iterator = get_paginated(
#         f"teams/seasons/{season_id}",
#         params={"select": "id"} # We only need the ID to make the link
#     )

#     count = 0
#     for team in iterator:
#         cur.execute(
#             """
#             INSERT INTO reference.season_teams (season_id, team_id, league_id, last_updated)
#             VALUES (%s, %s, %s, NOW())
#             ON CONFLICT (season_id, team_id) DO NOTHING
#             """,
#             (season_id, team.get("id"), league_id)
#         )
#         count += 1
    
#     conn.commit()
#     conn.close()
#     print(f"       -> Linked {count} teams.")


# # --- 2. THE DRIVER LOOP (Selects from DB and iterates) ---
# def sync_all_active_seasons():
#     conn = get_db()
#     cur = conn.cursor()

#     print("--- 1. Fetching Active Seasons from DB ---")
    
#     # ⚠️ IMPORTANT: We filter by 'is_current' to save API calls.
#     # If you want history, remove the WHERE clause or change it to specific IDs.
#     cur.execute("""
#         SELECT id, league_id 
#         FROM reference.seasons 
#     """)
    
#     # Load all rows into a list so we can close this DB connection
#     # before starting the long API process.
#     rows = cur.fetchall()
#     conn.close() 

#     print(f"--- 2. Found {len(rows)} seasons. Starting Loop ---")

#     for row in rows:
#         # row is a tuple: (season_id, league_id)
#         current_season_id = row[0]
#         current_league_id = row[1]

#         try:
#             sync_teams_in_season(current_season_id, current_league_id)
#         except Exception as e:
#             print(f"❌ Error on Season {current_season_id}: {e}")
#             # We continue the loop even if one fails
#             continue

#     print("\n✅ All Seasons Processed.")

# # --- RUN IT ---
# sync_all_active_seasons()

In [53]:
import requests
import json

# --- CONFIG ---
BASE_URL = "https://api.sportmonks.com/v3/football/fixtures"

# 1. TARGET STATS (Standard)
STATS_METRICS = {
    "accurate-crosses",
    "assists",
    "attacks",
    "ball-possession",
    "ball-safe",
    "big-chances-created",
    "big-chances-missed",
    "corners",
    "dangerous-attacks",
    "dribble-attempts",
    "duels-won",
    "fouls",
    "free-kicks",
    "goals",
    "hit-woodwork",
    "injuries",
    "interceptions",
    "long-passes",
    "offsides",
    "passes",
    "penalties",
    "saves",
    "shots-blocked",
    "shots-insidebox",
    "shots-off-target",
    "shots-on-target",
    "shots-outsidebox",
    "shots-total",
    "substitutions",
    "successful-headers",
    "successful-long-passes",
    "successful-long-passes-percentage",
    "successful-passes",
    "successful-passes-percentage",
    "tackles",
    "throwins",
    "total-crosses",
    "yellowcards",
}


def fetch_rich_stats(fixture_id):
    print(f"🚀 Fetching Rich Stats for Fixture {fixture_id}...")

    # KEY CHANGE: Added 'xGFixture' to includes
    includes = [
        "participants",  # To map Home/Away
        # "statistics.type",  # Standard Stats
        # 'timeline',
        # 'ballCoordinates'
        'xGFixture'
    ]

    url = f"{BASE_URL}/{fixture_id}"
    params = {"api_token": API_TOKEN, "include": ";".join(includes)}

    try:
        resp = requests.get(url, params=params)
        resp.raise_for_status()
        data = resp.json().get("data", {})
    except Exception as e:
        print(f"❌ API Error: {e}")
        return {}

    # --- 1. MAP TEAMS ---
    participants = data.get("participants", [])
    team_map = {}
    for p in participants:
        team_map[p["id"]] = p["meta"]["location"]  # 'home' or 'away'

    cleaned_stats = {}

    # --- 2. PARSE STANDARD STATS ---
    raw_stats = data.get("statistics", [])
    # print(raw_stats)
    # for s in raw_stats:
    #     code = s.get("type", {}).get("code")
    #     if code in STATS_METRICS:
    #         pid = s.get("participant_id")
    #         side = team_map.get(pid, "unknown")
    #         val = s.get("data", {}).get("value")

    #         if code not in cleaned_stats:
    #             cleaned_stats[code] = {"home": 0, "away": 0}
    #         cleaned_stats[code][side] = val

    # return cleaned_stats
    return data



TEST_ID = 19432471  # Gillingham vs Crawley
result = fetch_rich_stats(TEST_ID)


# Print xG first if it exists

# # Print the rest
# for k in sorted(result.keys()):
#     if k == "expected-goals":
#         continue
#     v = result[k]
#     print(f"{k:<30} | Home: {str(v.get('home')):<5} | Away: {str(v.get('away')):<5}")

🚀 Fetching Rich Stats for Fixture 19432471...


In [54]:
result

{'id': 19432471,
 'sport_id': 1,
 'league_id': 12,
 'season_id': 25649,
 'stage_id': 77476918,
 'group_id': None,
 'aggregate_id': None,
 'round_id': 372815,
 'state_id': 5,
 'venue_id': 339205,
 'name': 'Exeter City vs Wigan Athletic',
 'starting_at': '2025-11-08 15:00:00',
 'result_info': 'Game ended in draw.',
 'leg': '1/1',
 'details': None,
 'length': 90,
 'placeholder': False,
 'has_odds': True,
 'has_premium_odds': True,
 'starting_at_timestamp': 1762614000,
 'participants': [{'id': 185,
   'sport_id': 1,
   'country_id': 462,
   'venue_id': 339205,
   'gender': 'male',
   'name': 'Exeter City',
   'short_code': 'EXE',
   'image_path': 'https://cdn.sportmonks.com/images/soccer/teams/25/185.png',
   'founded': 1904,
   'type': 'domestic',
   'placeholder': False,
   'last_played_at': '2025-11-15 15:00:00',
   'meta': {'location': 'home', 'winner': False, 'position': 20}},
  {'id': 17,
   'sport_id': 1,
   'country_id': 462,
   'venue_id': 14,
   'gender': 'male',
   'name': 'Wiga

In [49]:
result['ballcoordinates']

[{'id': 256442434,
  'fixture_id': 19432471,
  'period_id': 6351118,
  'timer': '94:49',
  'x': '0.69',
  'y': '0.73'},
 {'id': 256442333,
  'fixture_id': 19432471,
  'period_id': 6351118,
  'timer': '94:44',
  'x': '0.69',
  'y': '0.73'},
 {'id': 256442248,
  'fixture_id': 19432471,
  'period_id': 6351118,
  'timer': '94:39',
  'x': '0.69',
  'y': '0.73'},
 {'id': 256442151,
  'fixture_id': 19432471,
  'period_id': 6351118,
  'timer': '94:33',
  'x': '0.69',
  'y': '0.73'},
 {'id': 256442085,
  'fixture_id': 19432471,
  'period_id': 6351118,
  'timer': '94:28',
  'x': '0.69',
  'y': '0.73'},
 {'id': 256441973,
  'fixture_id': 19432471,
  'period_id': 6351118,
  'timer': '94:23',
  'x': '0.69',
  'y': '0.73'},
 {'id': 256441881,
  'fixture_id': 19432471,
  'period_id': 6351118,
  'timer': '94:18',
  'x': '0.63',
  'y': '0.73'},
 {'id': 256441797,
  'fixture_id': 19432471,
  'period_id': 6351118,
  'timer': '94:14',
  'x': '0.63',
  'y': '0.73'},
 {'id': 256441692,
  'fixture_id': 19432

In [51]:
result['timeline']

[{'id': 152065549,
  'fixture_id': 19432471,
  'period_id': 6350549,
  'participant_id': 17,
  'type_id': 126,
  'section': 'timeline',
  'player_id': None,
  'related_player_id': None,
  'player_name': None,
  'related_player_name': None,
  'result': None,
  'info': None,
  'addition': '1st Corner',
  'minute': 8,
  'extra_minute': None,
  'injured': None,
  'on_bench': False,
  'coach_id': None,
  'sub_type_id': None,
  'detailed_period_id': 6350549,
  'rescinded': None,
  'sort_order': 1},
 {'id': 152066472,
  'fixture_id': 19432471,
  'period_id': 6350549,
  'participant_id': 17,
  'type_id': 126,
  'section': 'timeline',
  'player_id': None,
  'related_player_id': None,
  'player_name': None,
  'related_player_name': None,
  'result': None,
  'info': None,
  'addition': '2nd Corner',
  'minute': 22,
  'extra_minute': None,
  'injured': None,
  'on_bench': False,
  'coach_id': None,
  'sub_type_id': None,
  'detailed_period_id': 6350549,
  'rescinded': None,
  'sort_order': 2},
 {'

[{'id': 152066629,
  'fixture_id': 19432471,
  'period_id': 6350549,
  'participant_id': 17,
  'type_id': 14,
  'section': 'event',
  'player_id': 1481492,
  'related_player_id': 2905,
  'player_name': 'Callum Wright',
  'related_player_name': 'Morgan Fox',
  'result': '0-1',
  'info': 'Left foot shot',
  'addition': '1st Goal',
  'minute': 24,
  'extra_minute': None,
  'injured': None,
  'on_bench': False,
  'coach_id': None,
  'sub_type_id': 1521,
  'detailed_period_id': 6350549,
  'rescinded': None,
  'sort_order': 1},
 {'id': 152067183,
  'fixture_id': 19432471,
  'period_id': 6350549,
  'participant_id': 185,
  'type_id': 19,
  'section': 'event',
  'player_id': 37388639,
  'related_player_id': None,
  'player_name': 'Jayden Wareham',
  'related_player_name': None,
  'result': None,
  'info': 'Foul',
  'addition': '1st Yellowcard',
  'minute': 31,
  'extra_minute': None,
  'injured': None,
  'on_bench': False,
  'coach_id': None,
  'sub_type_id': None,
  'detailed_period_id': 6350

In [25]:
df = pd.DataFrame(result)
df.columns = [i.replace('-', '_') for i in df.columns]
df

,successful_passes_percentage,ball_possession,throwins,shots_total,duels_won,attacks,tackles,offsides,dangerous_attacks,passes,...,successful_long_passes,successful_long_passes_percentage,accurate_crosses,assists,yellowcards,big_chances_missed,shots_outsidebox,substitutions,ball_safe,injuries
home,86,48,19,15,42,101,10,0,59,493,...,16,47,4,3,2,2,2,5,86,0
away,88,52,7,8,42,109,23,3,49,521,...,26,57,1,0,2,0,4,5,64,3


In [ ]:
import time
import requests

# --- CONFIG ---
BASE_URL = "https://api.sportmonks.com/v3/football"


def sync_missing_stats_batch():
    conn = get_db()
    cur = conn.cursor()

    print("🔍 Scanning database for finished games with missing stats...")

    # 1. GET THE TO-DO LIST
    # We limit to 50 at a time to prevent rate-limit explosions during testing
    cur.execute(
        """
        SELECT f.id, f.name 
        FROM reference.fixtures f
        LEFT JOIN reference.match_stats ms ON f.id = ms.fixture_id
        WHERE ms.fixture_id IS NULL 
        AND f.starting_at < (NOW() - INTERVAL '3 hours') 
        AND f.starting_at > '2023-08-01' 
        ORDER BY f.starting_at DESC
        LIMIT 50;
    """
    )

    queue = cur.fetchall()

    if not queue:
        print("✅ No missing stats found! You are up to date.")
        return

    print(f"🚀 Found {len(queue)} games to process.")

    # 2. PROCESS THE QUEUE
    for row in queue:
        fixture_id = row[0]
        fixture_name = row[1]

        try:
            # Call the single sync function we wrote earlier
            sync_single_fixture_stats(fixture_id, cur)

            # Commit after every game so we don't lose progress if it crashes
            conn.commit()

            # ⚠️ BE POLITE TO THE API
            time.sleep(0.5)

        except Exception as e:
            print(f"❌ Failed on {fixture_name}: {e}")

    conn.close()
    print("✅ Batch Complete.")


# --- RE-USING YOUR FUNCTION (Slightly modified to accept cursor) ---
def sync_single_fixture_stats(fixture_id, cur):
    print(f"   Processing: {fixture_id} ...")

    url = f"{BASE_URL}/fixtures/{fixture_id}"
    params = {
        "api_token": API_TOKEN,
        "include": "participants;statistics.type;xGFixture",
    }

    resp = requests.get(url, params=params)
    if resp.status_code != 200:
        print(f"      ⚠️ API Error {resp.status_code}")
        return

    data = resp.json().get("data", {})

    # 1. Setup Containers
    match_rows = {}

    # 2. Map Teams (Home/Away)
    participants = data.get("participants", [])
    if not participants:
        print("      ⚠️ No participants data (Game might be postponed/cancelled)")
        return

    for p in participants:
        t_id = p["id"]
        match_rows[t_id] = {"is_home": (p["meta"]["location"] == "home"), "stats": {}}

    # 3. Map Stats
    STATS_MAP = {
        "goals": "goals",
        "expected-goals": "xg",
        "shots-total": "shots_total",
        "shots-on-target": "shots_on_target",
        "ball-possession": "possession",
        "passes": "passes",
        "corners": "corners",
        "fouls": "fouls",
        "yellowcards": "yellow_cards",
        "redcards": "red_cards",
    }

    # Fill Standard Stats
    for s in data.get("statistics", []):
        t_id = s["participant_id"]
        code = s.get("type", {}).get("code")
        val = s.get("data", {}).get("value")
        if t_id in match_rows and code in STATS_MAP:
            match_rows[t_id]["stats"][STATS_MAP[code]] = val

    # Fill xG (from expected include)
    for x in data.get("expected", []):
        t_id = x["participant_id"]
        val = x.get("data", {}).get("value")
        if t_id in match_rows:
            match_rows[t_id]["stats"]["xg"] = val

    # 4. Insert
    for t_id, row in match_rows.items():
        s = row["stats"]
        cur.execute(
            """
            INSERT INTO reference.match_stats (
                fixture_id, team_id, is_home, 
                goals, xg, shots_total, shots_on_target, 
                possession, passes, corners, fouls, 
                yellow_cards, red_cards, last_updated
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, NOW())
            ON CONFLICT (fixture_id, team_id) DO NOTHING;
            """,
            (
                fixture_id,
                t_id,
                row["is_home"],
                s.get("goals", 0),
                s.get("xg", 0.0),
                s.get("shots_total", 0),
                s.get("shots_on_target", 0),
                s.get("possession", 0),
                s.get("passes", 0),
                s.get("corners", 0),
                s.get("fouls", 0),
                s.get("yellow_cards", 0),
                s.get("red_cards", 0),
            ),
        )


# Run it
sync_missing_stats_batch()

... Fetching recent fixture IDs from Database
🚀 Starting Sync for 43497 fixtures...
   -> Saving stats for Fixture 19431936
   -> Saving stats for Fixture 19432498
   -> Saving stats for Fixture 19433035
   -> Saving stats for Fixture 19433041
   -> Saving stats for Fixture 19433037
   -> Saving stats for Fixture 19433038
   -> Saving stats for Fixture 19432480
   -> Saving stats for Fixture 19432481
   -> Saving stats for Fixture 19433031
   -> Saving stats for Fixture 19433032
   -> Saving stats for Fixture 19433033
   -> Saving stats for Fixture 19433034
   -> Saving stats for Fixture 19432483
   -> Saving stats for Fixture 19433036
   -> Saving stats for Fixture 19433039
   -> Saving stats for Fixture 19433040
   -> Saving stats for Fixture 19432485
   -> Saving stats for Fixture 19432486
   -> Saving stats for Fixture 19432488
   -> Saving stats for Fixture 19432490
   -> Saving stats for Fixture 19432484
   -> Saving stats for Fixture 19433042
   -> Saving stats for Fixture 19432

KeyboardInterrupt: 

In [20]:
result

{'successful-passes-percentage': {'home': 86, 'away': 88},
 'ball-possession': {'home': 48, 'away': 52},
 'throwins': {'home': 19, 'away': 7},
 'shots-total': {'home': 15, 'away': 8},
 'duels-won': {'home': 42, 'away': 42},
 'attacks': {'home': 101, 'away': 109},
 'tackles': {'home': 10, 'away': 23},
 'offsides': {'home': 0, 'away': 3},
 'dangerous-attacks': {'home': 59, 'away': 49},
 'passes': {'home': 493, 'away': 521},
 'shots-off-target': {'home': 4, 'away': 2},
 'free-kicks': {'home': 14, 'away': 8},
 'dribble-attempts': {'home': 27, 'away': 9},
 'successful-headers': {'home': 8, 'away': 7},
 'long-passes': {'home': 34, 'away': 46},
 'interceptions': {'home': 9, 'away': 5},
 'shots-blocked': {'home': 3, 'away': 1},
 'successful-passes': {'home': 425, 'away': 461},
 'corners': {'home': 9, 'away': 4},
 'saves': {'home': 4, 'away': 4},
 'total-crosses': {'home': 16, 'away': 20},
 'fouls': {'home': 8, 'away': 11},
 'shots-insidebox': {'home': 13, 'away': 4},
 'shots-on-target': {'home